# Convert annotated point cloud to a GeoTiff raster

The input for this notebok is the pre-processed and filtered cloud that results from notebook `1. Tree filter.ipynb`.

We create a raster with three bands:
1. `n_points`: the number of tree-points per cell
2. `hag`: the 80th percentile height above ground (HAG) for that cell
3. `nap`: the 80th percentile NAP height for that cell

This raster can be used for further analysis in standard GIS tools.

In [ ]:
import set_path

import numpy as np
import laspy
import pathlib
from tqdm.notebook import tqdm

from upcp.utils import clip_utils

import gvl.helper_functions as utils
import gvl.geotiff_utils as gt_utils

## Settings

In [ ]:
DATA_FOLDER = pathlib.Path('../data')

input_dir = DATA_FOLDER / 'ahn4_trees'
output_dir = DATA_FOLDER / 'ahn4_tree_grid'
N = 4  # Number of digits in tilecode format
TILE_WIDTH = 50  # Tile width in meters

EPSG = 28992     # RD coordinates
grid_res = 0.5   # 50cm resolution grid

include_overlap = False  # Add the existing overlap buffer around the tile
BUFFER = 5

# Our classification
UNKNOWN = 0
TREE = 1
NOISE = 2
OTHER = 0

In [ ]:
# Create output folder
output_dir.mkdir(parents=True, exist_ok=True)

## Main loop

In [ ]:
input_files = list(input_dir.glob('trees*.laz'))

In [ ]:
# Check existing output files (ignore this cell to re-run for all tiles)
existing_files = list(output_dir.glob('raster*.tif'))
existing_codes = {utils.get_tilecode_from_filename(file.name, n_digits=N)
                  for file in existing_files}

input_files = [file for file in input_files
               if utils.get_tilecode_from_filename(file.name, n_digits=N)
               not in existing_codes]

In [ ]:
pbar = tqdm(input_files, unit='file', smoothing=0)

for file in pbar:
    tilecode = utils.get_tilecode_from_filename(file.name, n_digits=N)
    pbar.set_postfix_str(tilecode)
    
    # Load LAS data
    las = laspy.read(file)
    mask = las.label == TREE
    if np.count_nonzero(mask) == 0:
        continue
    points_xyz = np.vstack((las.x[mask], las.y[mask], las.z[mask])).T
    hag = las.hag[mask]
    
    if include_overlap:
        ((x_min, y_max), (x_max, y_min)) = utils.get_bbox_from_tile_code(tilecode, padding=BUFFER, width=TILE_WIDTH, height=TILE_WIDTH)
        tilewidth = x_max - x_min  # Assume square tile
    else:
        ((x_min, y_max), (x_max, y_min)) = utils.get_bbox_from_tile_code(tilecode, padding=0, width=TILE_WIDTH, height=TILE_WIDTH)
        tilewidth = x_max - x_min  # Assume square tile
        mask = clip_utils.rectangle_clip(points_xyz, (x_min, y_min, x_max, y_max))
        if np.count_nonzero(mask) == 0:
            continue
        points_xyz = points_xyz[mask, :]
        hag = hag[mask]

    n_points, hag, nap = gt_utils.get_tile_grid(points_xyz, hag, x_min, y_min, tilewidth, grid_res)
    
    output_file = output_dir / f'raster_{tilecode}.tif'
    gt_utils.save_geotiff(output_file.as_posix(), n_points.T, hag.T, nap.T, x_min, y_min, grid_res, epsg=EPSG)